In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp gdrive/My\ Drive/Data.rar .

In [4]:
!unrar x -r Data.rar -idq



Would you like to replace the existing file Data/Test/image_id1.png
1675484 bytes, modified on 2019-03-01 18:58
with a new one
1675484 bytes, modified on 2019-03-01 18:58

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit A


In [5]:
from os import listdir
from os.path import isfile, join
fakeimagepaths='Data/Training/Fake/Images'
listIDs = [['Fake/Images/'+f,1,'Fake/Masks/'+f[:-4]+'.mask.png'] for f in listdir(fakeimagepaths) if isfile(join(fakeimagepaths, f))]
pristinepaths='Data/Training/Pristine'
listIDs1=[['Pristine/'+f,0] for f in listdir(pristinepaths) if isfile(join(pristinepaths, f))]
for ele in listIDs1:
    listIDs.append(ele)
from random import shuffle
shuffle(listIDs)
print(listIDs)
trainIDs=listIDs[:int(len(listIDs)*0.8)]
valIDs=listIDs[int(len(listIDs)*0.8):]

[['Pristine/0cff314c97cc71a8829c9634033c7b5d.png', 0], ['Pristine/5b1086f364fc04a7618d93c0e1c01b42.png', 0], ['Pristine/03cf7a8ef349c3b618f95a85093568db.png', 0], ['Pristine/6bc84a0ca7f67411a8297fb588904e5a.png', 0], ['Fake/Images/ab3e3e6a64d42b2ec53685f24192daf3.png', 1, 'Fake/Masks/ab3e3e6a64d42b2ec53685f24192daf3.mask.png'], ['Pristine/4a7ab4c11dc562171cc0424372223830.png', 0], ['Fake/Images/2564876058a684e6d899f3575081b16f.png', 1, 'Fake/Masks/2564876058a684e6d899f3575081b16f.mask.png'], ['Pristine/1c5d35eeabb73e42cbfde3b45d932a7a.png', 0], ['Fake/Images/fdf110fa789efa05114811412f54b2a9.png', 1, 'Fake/Masks/fdf110fa789efa05114811412f54b2a9.mask.png'], ['Pristine/2abfaf669584770c4bb2a43cc50b2ecd.png', 0], ['Pristine/6d1a4081b063504792374d75ea12e708.png', 0], ['Pristine/2cf8f4cf134b51a869e8dc28af2110aa.png', 0], ['Pristine/2e24d9377b08dd0ae207f7d6bd36753f.png', 0], ['Fake/Images/cdce0701fa96db632af5facf074fe654.png', 1, 'Fake/Masks/cdce0701fa96db632af5facf074fe654.mask.png'], ['Fake/

In [6]:
print(trainIDs)
print(len(trainIDs))
print(valIDs)
print(len(valIDs))

[['Pristine/0cff314c97cc71a8829c9634033c7b5d.png', 0], ['Pristine/5b1086f364fc04a7618d93c0e1c01b42.png', 0], ['Pristine/03cf7a8ef349c3b618f95a85093568db.png', 0], ['Pristine/6bc84a0ca7f67411a8297fb588904e5a.png', 0], ['Fake/Images/ab3e3e6a64d42b2ec53685f24192daf3.png', 1, 'Fake/Masks/ab3e3e6a64d42b2ec53685f24192daf3.mask.png'], ['Pristine/4a7ab4c11dc562171cc0424372223830.png', 0], ['Fake/Images/2564876058a684e6d899f3575081b16f.png', 1, 'Fake/Masks/2564876058a684e6d899f3575081b16f.mask.png'], ['Pristine/1c5d35eeabb73e42cbfde3b45d932a7a.png', 0], ['Fake/Images/fdf110fa789efa05114811412f54b2a9.png', 1, 'Fake/Masks/fdf110fa789efa05114811412f54b2a9.mask.png'], ['Pristine/2abfaf669584770c4bb2a43cc50b2ecd.png', 0], ['Pristine/6d1a4081b063504792374d75ea12e708.png', 0], ['Pristine/2cf8f4cf134b51a869e8dc28af2110aa.png', 0], ['Pristine/2e24d9377b08dd0ae207f7d6bd36753f.png', 0], ['Fake/Images/cdce0701fa96db632af5facf074fe654.png', 1, 'Fake/Masks/cdce0701fa96db632af5facf074fe654.mask.png'], ['Fake/

In [7]:
import numpy as np
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
import random
import tensorflow as tf
import tensorflow
from PIL import Image
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs,ROOT,num_classes=1,batch_size=8, height=512,width=512, n_channels=3,
                 shuffle=False,is_training=False,min_scale=1,max_scale=1):
        'Initialization'
        self.height=height
        self.width=width
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.ROOT=ROOT
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.is_training=is_training
        self.min_scale=min_scale
        self.max_scale=max_scale
        self.on_epoch_end()
        self.num_classes=num_classes

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.ndarray((self.batch_size, self.height,self.width, self.n_channels), dtype=np.float32)
        y = np.ndarray((self.batch_size,self.height,self.width,self.num_classes), dtype=np.uint8)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            
            image=load_img(self.ROOT+ID[0],target_size=(self.height,self.width))
            image=img_to_array(image)
            
            if(ID[1]!=0):
                label=load_img(self.ROOT+ID[2],target_size=(self.height,self.width),grayscale=True)
                label=img_to_array(label)
                label=label/255.0
            else:
                label=np.ones(shape=(self.height,self.width,1))
           
            X[i,]=image/255.0

            y[i,] = label

        return X, y


Using TensorFlow backend.


In [0]:
ROOT='Data/Training/'
params = {'height': 1024,
          'width': 800,
          'batch_size': 2,
          'num_classes':1,
          'n_channels': 3,
          'ROOT': ROOT,
          'shuffle': True,
          'is_training':True}
training_generator = DataGenerator(listIDs, **params)
params = {'height': 1024,
          'width': 800,
          'batch_size': 1,
          'n_channels': 3,
          'num_classes':1,
          'ROOT': ROOT,
          'shuffle': True,
          'is_training':True}
validation_generator = DataGenerator(valIDs, **params)

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Conv2D,Conv2DTranspose,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras import backend as K


num_classes=1

vgg16=VGG16(include_top=False,weights='imagenet',input_shape=(1024, 800, 3),classes=1000)

for layer in vgg16.layers:
	layer.trainable=False


model=vgg16.output


model=Conv2D(256, (3, 3), activation='relu', padding='same')(model)
model_tr=Conv2DTranspose(256,(3,3),strides=2,padding='same',activation='relu')(model)
model=concatenate([vgg16.layers[17].output,model_tr],axis=3)

model=Conv2D(256, (3, 3), activation='relu', padding='same')(model)
model_tr=Conv2DTranspose(256,(3,3),strides=2,padding='same',activation='relu')(model)
model=concatenate([vgg16.layers[13].output,model_tr],axis=3)

model=Conv2D(128, (3, 3), activation='relu', padding='same')(model)
model_tr=Conv2DTranspose(128,(3,3),strides=2,padding='same',activation='relu')(model)
model=concatenate([vgg16.layers[9].output,model_tr],axis=3)

model=Conv2D(64, (3, 3), activation='relu', padding='same')(model)
model_tr=Conv2DTranspose(64,(3,3),strides=2,padding='same',activation='relu')(model)
model=concatenate([vgg16.layers[5].output,model_tr],axis=3)

model=Conv2D(32, (3, 3), activation='relu', padding='same')(model)
model_tr=Conv2DTranspose(32,(3,3),strides=2,padding='same',activation='relu')(model)
model=concatenate([vgg16.layers[2].output,model_tr],axis=3)

model=Conv2D(32, (3, 3), activation='relu', padding='same')(model)
model=Conv2D(num_classes, (1, 1), activation='sigmoid', padding='same')(model)
vgg16Unet = Model(inputs = vgg16.input, outputs = model)
vgg16Unet.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['acc'])


In [13]:
vgg16Unet.fit_generator(generator=training_generator,validation_data=validation_generator,epochs=10)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:98: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


Epoch 1/10
 79/725 [==>...........................] - ETA: 15:39 - loss: 0.2776 - acc: 0.9766

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:885: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


725/725 [==============================] - 1061s 1s/step - loss: 0.2073 - acc: 0.9805 - val_loss: 0.0985 - val_acc: 0.9817
Epoch 2/10
725/725 [==============================] - 1041s 1s/step - loss: 0.1977 - acc: 0.9807 - val_loss: 0.0917 - val_acc: 0.9817
Epoch 3/10
711/725 [============================>.] - ETA: 17s - loss: 0.2007 - acc: 0.9806

KeyboardInterrupt: ignored

In [0]:
vgg16Unet.save('gdrive/My Drive/first_manip.h5')

In [0]:
vgg16Unet.fit_generator(generator=training_generator,epochs=10)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:98: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


163/725 [=====>........................] - ETA: 19:15 - loss: 0.7674 - acc: 0.8466

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:885: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


725/725 [==============================] - 1489s 2s/step - loss: 0.7436 - acc: 0.8551
Epoch 2/10
725/725 [==============================] - 1485s 2s/step - loss: 0.6966 - acc: 0.8653
Epoch 3/10
725/725 [==============================] - 1487s 2s/step - loss: 0.6626 - acc: 0.8760
Epoch 4/10
725/725 [==============================] - 1487s 2s/step - loss: 0.6396 - acc: 0.8808
Epoch 5/10
725/725 [==============================] - 1485s 2s/step - loss: 0.6106 - acc: 0.8858
Epoch 6/10
725/725 [==============================] - 1488s 2s/step - loss: 0.5885 - acc: 0.8917
Epoch 7/10
725/725 [==============================] - 1490s 2s/step - loss: 0.5480 - acc: 0.9005
Epoch 8/10
725/725 [==============================] - 1503s 2s/step - loss: 0.5320 - acc: 0.9037
Epoch 9/10
725/725 [==============================] - 1503s 2s/step - loss: 0.5067 - acc: 0.9089
Epoch 10/10
725/725 [==============================] - 1503s 2s/step - loss: 0.4798 - acc: 0.9131


In [0]:
vgg16Unet.save('gdrive/My Drive/first_manip.h5')

In [0]:
vgg16Unet.fit_generator(generator=training_generator,epochs=10)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:98: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


169/725 [=====>........................] - ETA: 19:13 - loss: 0.4461 - acc: 0.9221

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:885: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


725/725 [==============================] - 1503s 2s/step - loss: 0.4601 - acc: 0.9178
Epoch 2/10
725/725 [==============================] - 1502s 2s/step - loss: 0.4675 - acc: 0.9180
Epoch 3/10
725/725 [==============================] - 1504s 2s/step - loss: 0.4191 - acc: 0.9276
Epoch 4/10
725/725 [==============================] - 1505s 2s/step - loss: 0.4177 - acc: 0.9274
Epoch 5/10
725/725 [==============================] - 1482s 2s/step - loss: 0.4022 - acc: 0.9296
Epoch 6/10
725/725 [==============================] - 1470s 2s/step - loss: 0.3764 - acc: 0.9347
Epoch 7/10
725/725 [==============================] - 1471s 2s/step - loss: 0.3673 - acc: 0.9372
Epoch 8/10
725/725 [==============================] - 1497s 2s/step - loss: 0.3564 - acc: 0.9380
Epoch 9/10
426/725 [================>.............] - ETA: 10:04 - loss: 0.3392 - acc: 0.9422

In [0]:
vgg16Unet.save('gdrive/My Drive/first_manip.h5')

In [0]:
!cd Data && mkdir Test_masks

In [0]:
testlistIDs = [f for f in listdir('Data/Test') if isfile(join('Data/Test', f))]

In [0]:
for ID in testlistIDs:
    image=load_img('Data/Test/'+ID,target_size=(1024,800))
    image=img_to_array(image)
    image=image/255.0
    image=np.reshape(image,(1,1024,800,3))
    mask=vgg16Unet.predict(image,batch_size=1)
    mask[mask<=0.8]=0
    mask[mask>0.8]=255
    mask=np.reshape(mask,(1024,800,1))
    pil_image = array_to_img(mask)
    pil_image=pil_image.resize((1024,786))
    with tf.gfile.Open('Data/Test_masks/'+ID,mode='w') as f:
        pil_image.save(f, 'PNG')
    
    

In [27]:
!cd Data && zip -r Test_masks.zip Test_masks

updating: Test_masks/ (stored 0%)
updating: Test_masks/image_id20.png (deflated 95%)
updating: Test_masks/image_id13.png (deflated 95%)
updating: Test_masks/image_id33.png (deflated 95%)
updating: Test_masks/image_id11.png (deflated 95%)
updating: Test_masks/image_id1.png (deflated 95%)
updating: Test_masks/image_id47.png (deflated 95%)
updating: Test_masks/image_id7.png (deflated 95%)
updating: Test_masks/image_id50.png (deflated 95%)
updating: Test_masks/image_id45.png (deflated 95%)
updating: Test_masks/image_id48.png (deflated 95%)
updating: Test_masks/image_id14.png (deflated 95%)
updating: Test_masks/image_id12.png (deflated 95%)
updating: Test_masks/image_id9.png (deflated 95%)
updating: Test_masks/image_id31.png (deflated 95%)
updating: Test_masks/image_id16.png (deflated 95%)
updating: Test_masks/image_id19.png (deflated 95%)
updating: Test_masks/image_id18.png (deflated 95%)
updating: Test_masks/image_id6.png (deflated 95%)
updating: Test_masks/image_id2.png (deflated 95%)
up